<a href="https://colab.research.google.com/github/ShaheenPafiast/-simple-ai-python-projects/blob/main/Simple_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!git clone https://github.com/Adiprogrammer7/intent_based_chatbot.git
%cd intent_based_chatbot
!ls


Cloning into 'intent_based_chatbot'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 76 (delta 34), reused 67 (delta 29), pack-reused 0 (from 0)
Receiving objects: 100% (76/76), 208.34 KiB | 1.67 MiB/s, done.
Resolving deltas: 100% (34/34), done.
/content/intent_based_chatbot/intent_based_chatbot/intent_based_chatbot/intent_based_chatbot
chatbot_model.tflearn.data-00000-of-00001  README.md
chatbot_model.tflearn.index		   requirements.txt
chatbot_model.tflearn.meta		   saved_variables.pickle
checkpoint				   training_chatbot.ipynb
intents.json				   utils.py
main.py


In [19]:
!pip -q install tensorflow nltk


In [20]:
import nltk

# Core resources needed for word_tokenize + lemmatizer
nltk.download("punkt")
nltk.download("punkt_tab")     # newer NLTK sometimes needs this
nltk.download("wordnet")
nltk.download("omw-1.4")       # WordNet helper


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [21]:
import json, random, pickle, numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import layers, models

lemmatizer = WordNetLemmatizer()

# ---- Load intents.json safely ----
with open("intents.json", "r", encoding="utf-8") as f:
    intents = json.load(f)

if "intents" not in intents or not isinstance(intents["intents"], list):
    raise ValueError("❌ intents.json must contain a top-level key 'intents' as a list.")

words, labels, docs_x, docs_y = [], [], [], []

ignore = set(list("?!.,;:()'\"-"))

for intent in intents["intents"]:
    tag = intent.get("tag", None)
    patterns = intent.get("patterns", [])
    responses = intent.get("responses", [])

    # minimal validation
    if not tag or not isinstance(patterns, list) or not isinstance(responses, list):
        continue

    if tag not in labels:
        labels.append(tag)

    for pattern in patterns:
        if not isinstance(pattern, str) or not pattern.strip():
            continue

        tokens = nltk.word_tokenize(pattern)
        tokens = [t.lower() for t in tokens if t not in ignore]
        tokens = [lemmatizer.lemmatize(t) for t in tokens]

        if len(tokens) == 0:
            continue

        words.extend(tokens)
        docs_x.append(tokens)
        docs_y.append(tag)

# ---- Hard stop if no data ----
if len(docs_x) == 0:
    raise ValueError("❌ No training patterns found. Check intents.json -> intents -> patterns.")

words = sorted(list(set(words)))
labels = sorted(list(set(labels)))

print("✅ Samples:", len(docs_x))
print("✅ Vocab size:", len(words))
print("✅ Labels:", labels)

label_to_idx = {lab: i for i, lab in enumerate(labels)}

# ---- Build BoW training vectors ----
X, y = [], []
for tokens, tag in zip(docs_x, docs_y):
    bow = [1.0 if w in tokens else 0.0 for w in words]
    X.append(bow)

    out = [0.0] * len(labels)
    out[label_to_idx[tag]] = 1.0
    y.append(out)

X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

# ---- TF2 Model ----
tf.random.set_seed(42)

model = models.Sequential([
    layers.Input(shape=(len(words),)),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(len(labels), activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(X, y, epochs=200, batch_size=8, verbose=0)
final_acc = float(history.history["accuracy"][-1])
print(f"✅ Training complete. Final accuracy: {final_acc:.3f}")

# ---- Save everything ----
model.save("chatbot_tf2.keras")
with open("tf2_assets.pkl", "wb") as f:
    pickle.dump({"words": words, "labels": labels, "intents": intents}, f)

print("✅ Saved: chatbot_tf2.keras + tf2_assets.pkl")


✅ Samples: 34
✅ Vocab size: 59
✅ Labels: ['address', 'booking', 'goodbye', 'greeting', 'multiple_rooms', 'name', 'one_room', 'thanks']
✅ Training complete. Final accuracy: 1.000
✅ Saved: chatbot_tf2.keras + tf2_assets.pkl


In [22]:
import pickle, numpy as np, random
import nltk
from nltk.stem import WordNetLemmatizer
import tensorflow as tf

lemmatizer = WordNetLemmatizer()
ignore = set(list("?!.,;:()'\"-"))

with open("tf2_assets.pkl", "rb") as f:
    data = pickle.load(f)

words = data["words"]
labels = data["labels"]
intents = data["intents"]
model = tf.keras.models.load_model("chatbot_tf2.keras")

def clean_sentence(text: str):
    tokens = nltk.word_tokenize(text)
    tokens = [t.lower() for t in tokens if t not in ignore]
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return tokens

def make_bow(tokens):
    vec = np.zeros(len(words), dtype=np.float32)
    token_set = set(tokens)
    for i, w in enumerate(words):
        if w in token_set:
            vec[i] = 1.0
    return vec.reshape(1, -1)

def find_responses(tag):
    for intent in intents["intents"]:
        if intent.get("tag") == tag:
            res = intent.get("responses", [])
            if isinstance(res, list) and len(res) > 0:
                return res
    return ["I don't have a response for that yet."]

def chatbot_reply(user_text, threshold=0.55):
    tokens = clean_sentence(user_text)
    if len(tokens) == 0:
        return "Please type something 🙂", None, 0.0

    probs = model.predict(make_bow(tokens), verbose=0)[0]
    idx = int(np.argmax(probs))
    conf = float(probs[idx])
    tag = labels[idx]

    if conf < threshold:
        return "Sorry, I didn't understand. Try asking in a different way.", None, conf

    return random.choice(find_responses(tag)), tag, conf

print("✅ Chatbot running (type: quit to stop)\n")

while True:
    user = input("You: ").strip()
    if user.lower() in {"quit", "exit"}:
        print("Bot: Bye!")
        break

    ans, tag, conf = chatbot_reply(user, threshold=0.55)
    if tag is None:
        print(f"Bot: {ans}  (conf={conf:.2f})")
    else:
        print(f"Bot ({tag}, conf={conf:.2f}): {ans}")


✅ Chatbot running (type: quit to stop)

You: hello
Bot (greeting, conf=1.00): Hello! Good to see you!
You: how are you
Bot (greeting, conf=1.00): Hey genius, what's up?
You: good
Bot (goodbye, conf=0.94): Goodbye! Have a nice day.
You: Tomorro I have paper of NLP
Bot: Sorry, I didn't understand. Try asking in a different way.  (conf=0.49)
You: i have paper tomorro
Bot (goodbye, conf=0.83): See you soon pal...
You: why you talk with me like this
Bot (thanks, conf=0.60): Happy to help!
You: ok 
Bot: Sorry, I didn't understand. Try asking in a different way.  (conf=0.43)
You: tomorrow is my Final Exam
Bot (address, conf=0.59): Our hotel is located at 'Malibu Point 10880, 90265' :)
You: Quit to stop
Bot: Sorry, I didn't understand. Try asking in a different way.  (conf=0.47)
You: Quit
Bot: Bye!
